In [1]:
import findspark
findspark.init()
import pyspark
import numpy as np

sc = pyspark.SparkContext(appName="Spark1")

In [2]:
# loading the data
lines = sc.textFile('Shakespeare.txt')

In [3]:
# loading the data
lines = sc.textFile('Shakespeare.txt')
words = lines.flatMap(lambda x:x.split(" "))

# Filtering the words
words = words.filter(lambda x: sum([not v.isupper() for v in x[:-1]]) and
                     sum([not v.isdigit() for v in x[:]]))

# Set the rdd in the form (correct_word, index)
words = words.zipWithIndex().map(lambda x: (x[1], x[0]))

In [9]:
occurence_words.collect()

[((u'should', u'not.'), [(u'This', 1), (u'Deceiving', 1), (u'She', 1)]),
 ((u"worship's", u'friend,'), [(u'and', 1)]),
 ((u'art', u"stall'd"), [(u'in', 1)]),
 ((u'Exit', u'Hast'), [(u'thou', 1)]),
 ((u'to', u'Antipholus'), [(u'of', 2)]),
 ((u'prevailed?', u'me,'), [(u'but', 1)]),
 ((u'Therefore,', u'as'), [(u'you', 1), (u'am', 1), (u'he', 1)]),
 ((u'ill.', u'Neither.'), [(u'What,', 1)]),
 ((u'less', u'loyal,'), [(u'Nearer', 1)]),
 ((u'render', u'what'), [(u'we', 1)]),
 ((u'do,', u'well'), [(u'know', 1)]),
 ((u'Where', u'intend'), [(u'holy', 1)]),
 ((u'pause.', u'Myself'), [(u'the', 1)]),
 ((u'ring', u'again.'), [(u'Plutus', 1)]),
 ((u'sight', u'For'), [(u'terror,', 1), (u'fear', 1)]),
 ((u'Indeed,', u'this'), [(u'counsellor', 1)]),
 ((u'parallel', u'fellow'), [(u'to', 1)]),
 ((u'fool,', u'boy?'), [(u'Fool.', 1)]),
 ((u'offense', u'in'), [(u'us,', 1)]),
 ((u"expos'd,", u'my'), [(u'lord,', 1)]),
 ((u'your', u'title'), [(u'to', 2)]),
 ((u'shall', u'drown'), [(u'the', 1)]),
 ((u'forsooth;'

In [5]:
# Change the index to group with the same index words triplet
first_words = words.map(lambda x: (x[0] + 2, x[1]))
second_words = words.map(lambda x: (x[0] + 1, x[1]))

In [6]:
double_words = first_words.join(second_words).map(lambda x: (x[0], tuple(x[1])))

In [7]:
triplet_words = words.join(double_words).map(lambda x: (list(x[1])[1],
                                             list(x[1])[0]))

In [8]:
def count_list(x):
    key = x[0]
    value = list(x[1])
    counter = [(w, value.count(w)) for w in list(set(value))]
    return key, counter

# Building the asked rdd
occurence_words = triplet_words.groupByKey().map(count_list)

In [22]:
# ### Text generation


def next_word(counter):
    '''
    Return the next word based on the next word transition given in counter.
    The probability of transition is computed on the square of the transition
    to introduce a bias on the count.
    '''
    t = [c[1] for c in counter]
    t2 = np.multiply(t, t)
    transition_proba = (1. / sum(t2)) * t2
    next_stage = np.random.multinomial(1, transition_proba)

    return counter[next_stage.argmax()][0]


def Shakespeare_quill(occurence_graph, s=10, w=20):
    '''
    Function to make Shakespeare's quill alive, generate s sentences of w words
    Args:
    s: number of sentences
    w: number of words per sentence
    Return:
    words_list: the list of the words
    '''
    # current: current double word
    # counter: occurences of the next word after current
    current, counter = occurence_graph.takeSample(True, 1)[0]
    words_list = list(current)

    for i in xrange(0, s):
        for j in xrange(0, w):
            # Compute the next word
            word = next_word(counter)
            # Debug mode
            # print('newt word is {}'.format(word))
            # Add the next word to the current line
            words_list.append(word)
            # Set the next tuple
            current = (current[1], word)
            # Debug mode
            # print('current word is {}'.format(current))
            # Retrieve the counter information of the next tuple
            counter = occurence_graph.lookup(current)[0]
            # Debug mode
            # print('counter is {}'.format(counter))
    return words_list

In [23]:
words_list = Shakespeare_quill(occurence_words)


In [24]:
def print_text(words_list, s=10, w=20):
    for i in xrange(0, s):
        print(' '.join(words_list[i * w:(i+1)*w]))

In [27]:
print_text(words_list, s=10)

creation. it true, Salerio? Hath all so long as live. Nay, told you enough of your most noble father. Gentlemen,
Heaven hath through me restor'd the King of France our shrouds and tacklings? And, though unskilful, why not death on't,
would adventure. Well then, the people, and The troop is past. Brother, my lord is cold And will, no doubt,
my lord. Ham. mean, my lord? Ham. Why, let the world but you; Nor now my father hath no more
to be of more grave aspect. think not on him that did debate this matter little Cupid's crafty arrow made,
That millions of strange shapes, of habits, and of good fellowship come to him. Plantagenet, for all the world, man;
see'st not? The lords are all supplied and fill'd, Her sweet perfections, with one consent praise new-born gawds, Though they
themselves be bevel; their rank thoughts, my gentle cousin, were it but to raise up him. Ben. Come, knock and
can; he'll speak with you. You fillip me o' th' tongue, Consuming it with your sprites; you're pow'rfull 